<a href="https://colab.research.google.com/github/Nancy-Shi/Networks_2_Vaccines/blob/main/091123_Vaccine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2-Layer Vaccine Model with Informtion, Cognition/Epidemic

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import math as math
from math import log
import seaborn as sns
import pandas as pd
import matplotlib.ticker as ticker

In [12]:
# Generate Degree Sequence
def generate_degree_sequence(n, gamma, kmin):
    # Generate a random set from the power law distribution
    u = np.random.uniform(size=n)
    degrees = np.ceil((1.0 - u) ** (-1.0 / (gamma - 1.0)))

    # Adjust degrees based on the minimum and maximum degree values
    kmax = int(np.sqrt(n))
    #kmax = int(((gamma-1)/(gamma-2) * n )** (1/gamma))  # max degree
    # kmax = int(1.5*n**(1/4)) # max degree allowed is 1.5*n^(1/4)
    degrees = degrees[(degrees >= kmin) & (degrees <= kmax)].astype(int)

    # Truncate or pad the sequence to match the length specified
    if len(degrees) >= n:
        degrees = degrees[:n]
    else:
        degrees = np.concatenate((degrees, np.full(n - len(degrees), kmin)))

    return degrees.tolist()

In [13]:
# Assign Thresholds
# Defines the parameters to be used
mu = 0.1
sigma = 0.05

# Function to assign thresholds to the individual nodes
def assign_thresholds(hypergraph, mu, sigma):
    NV = hypergraph.order()
    Ltre = {}

    for node in hypergraph.nodes():
          # Uniform distribution: #
          #Ltre[node] = np.random.uniform()
          # Normal distrution
          while True:
              threshold = random.gauss(mu, sigma)
              if 0 < threshold < 1:
                  break
          Ltre[node] = threshold

    return Ltre

In [25]:
def Vaccine_model(inw, ldeg_i, ltre, enw, ldeg_e, lam, alp, zeta_1, zeta_2, zeta_info, zeta_epi, beta_1, beta_2, beta_3, mu, n_sample):

  t_max = 100000      # Set maximum time
  kmax_i = max (ldeg_i)     # Get maximum hyperedge degree in information layer
  kmax_e = max (ldeg_e)     # Get maximum degree in epidemic layer
  N = inw.order()  # Get the network size

  rho_C = []   # Keep track of fraction of corrected in information layer
  rho_V1 = []   # Keep track of fraction of vaccinated with vaccine type 1 in epidemic layer
  rho_V2 = []   # Keep track of fraction of vaccinated with vaccine type 2 in epidemic layer
  rho_R = []   # Keep track of fraction of recovered in epidemic layer

  for i_samp in range(1, n_sample + 1):
      t = 0                 # Initialize time, number of corrected, number of recovered
      N_corrected = 0
      N_recovered = 0

      info_states = {j: "U" for j in inw.nodes()}   # Initialize information and disease states
      disease_states = {k: "S" for k in enw.nodes()}

      gossip = []     # Create lists to store gossip and corrected individuals in information layer
      corrected = []

      rumor_node_0 = np.random.choice(list(inw.nodes()))   # Pick a random person to start misinformaiton spreading
      info_states[rumor_node_0] = "G"
      gossip.append(rumor_node_0)
      N_gossip = 1
      N_e_i = inw.degree(rumor_node_0)


      susceptible = list(enw.nodes())
      infected = []     # Create lists to store infected and recovered individuals in epidemic layer
      recovered = []
      vaccinated_first = []
      vaccinated_second = []
      N_vaccine_1 = 0
      N_vaccine_2 = 0

      ill_node_0 = np.random.choice(list(enw.nodes()))   # Pick a random person to start disease spreading
      disease_states[ill_node_0] = "I"
      infected.append(ill_node_0)
      N_infected = 1
      N_e_e = enw.degree(ill_node_0)

      while N_gossip > 0:   # We stop when there is no infection and no gossip
          total_rate = lam * N_e_i + 2 * alp * N_e_i + beta_1 * N_e_e + mu * N_infected + zeta_info * (N-N_gossip) + zeta_epi * (N-N_vaccine_1)
          tau = -np.log(np.random.uniform(1e-6, 1)) / total_rate
          t += tau

          if t >= t_max:
                break

          # Determine which event occurs
          event = np.random.uniform()
          p1 = (lam * N_e_i) / total_rate     # rumor spreading
          p2 = (lam * N_e_i + alp * N_e_i) / total_rate  # rumor stifling (by meeting stifling neighbor threshold)
          p3 = (lam * N_e_i + 2 * alp * N_e_i) / total_rate  # rumor stifling (by meeting gossip neighbor threshold)

          p4 = (lam * N_e_i + 2 * alp * N_e_i + beta_1 * N_e_e) / total_rate  # disease propagation
          p5 = (lam * N_e_i + 2 * alp * N_e_i + beta_1 * N_e_e + mu * N_infected) / total_rate  # disease recovery

          p6 = (lam * N_e_i + 2 * alp * N_e_i + beta_NN * N_e_e + mu * N_infected + zeta_info * (N-N_gossip)) / total_rate # get vaccine 1 by information
          p7 = (lam * N_e_i + 2 * alp * N_e_i + beta_NN * N_e_e + mu * N_infected + zeta_info * (N-N_gossip) + zeta_epi * (N-N_vaccine_1)) / total_rate # get vaccine 1 by neighbourhood

          # Determine if accept selected individual based on degree distribution
          q_deg_i = np.random.uniform()
          q_deg_e = np.random.uniform()

          # Case 1: Rumor spreading (Pairwise Threshold)
          if event < p1:
                gossip_node = random.choice(gossip)
                if q_deg_i < inw.degree(gossip_node) / kmax_i:
                    neighbors = inw.neighbors[gossip_node]
                    neighbor = random.choice(neighbors)
                    if info_states[neighbor] == "U":
                          count_gossip_neighbors = sum(1 for node in inw.neighbors(neighbor) if info_states[node] == "G")
                          if count_gossip_neighbors / len(inw.neighbors(neighbor)) >= ltre[neighbor]:
                                info_states[neighbor] = "G"  # uninformed neighbor becomes gossip spreader
                                gossip.append(neighbor)
                                N_gossip += 1

          # Case 2: Rumor stifling (by meeting corrected stiflers) (Pairwise Threshold)
          elif event < p2:
            #if N_gossip > 0:
                stifler_node = np.random.choice(gossip)
                if q_deg_i < inw.degree(stifler_node)  / kmax_i:
                    neighbors = inw.neighbors[stifler_node]
                    count_stifler_neighbors = sum(1 for node in inw.neighbors(stifler_node) if info_states[node] == "C")
                    if count_stifler_neighbors / len(neighbors) >= ltre[stifler_node]:
                            info_states[stifler_node] = "C"
                            N_gossip -= 1
                            gossip.remove(stifler_node)
                            corrected.append(stifler_node)
                            N_corrected += 1

          # Case 3: Rumor stifling (by meeting gossip spreaders) (Pairwise Threshold)
          elif event < p3:
            #if N_gossip > 0:
                stifler_node = np.random.choice(gossip)
                if q_deg_i < inw.degree(stifler_node) / kmax_i:
                    neighbors = inw.neighbors[stifler_node]
                    count_gossip_neighbors = sum(1 for node in inw.neighbors(stifler_node) if info_states[node] == "G")
                    if count_gossip_neighbors / len(neighbors) >= ltre[stifler_node]:
                            info_states[stifler_node] = "C"
                            N_gossip -= 1
                            gossip.remove(stifler_node)
                            corrected.append(stifler_node)
                            N_corrected += 1

          # Case 4: Disease propagation
          elif event < p4:
            if N_infected > 0:
              infected_node = np.random.choice(infected)
              neighbors = list(enw.neighbors(infected_node))
              susceptible_or_vaccinated_neighbors = [n for n in neighbors if (disease_states[n] == "S" or disease_states[n] == "V1" or disease_states[n] == "V2")]

              if len(susceptible_or_vaccinated_neighbors) > 0:
                  neighbor = np.random.choice(susceptible_or_vaccinated_neighbors)

                  # Determine the appropriate transmission rate based on protection status
                  if disease_states[neighbor] == "V1":
                            transmission_rate = beta_2/beta_1
                  elif disease_states[neighbor] == "V2":
                            transmission_rate = beta_3/beta_1
                  else:
                            transmission_rate = beta_1/beta_1

                  if np.random.uniform() < transmission_rate:
                      disease_states[neighbor] = "I"
                      infected.append(neighbor)
                      N_infected += 1
                      N_e_e += enw.degree(neighbor)

          # Case 5: Disease recovery
          elif event < p5:
            if N_infected > 0:
                recovered_node = np.random.choice(infected)
                if q_deg_e < ldeg_e[recovered_node]/kmax_e:
                    disease_states[recovered_node] = "R"
                    infected.remove(recovered_node)
                    recovered.append(recovered_node)
                    N_infected -= 1
                    N_recovered += 1
                    N_e_e -= enw.degree(recovered_node)

          # Case 6: Get vaccine 1 based on information layer
          # rate = zeta_info * (1 - n_G / k_info)
          elif event < p6:
            if len(susceptible) > 0:
                node_to_vaccinate = np.random.choice(susceptible)
                n_G = sum(1 for node in filter(lambda x: info_states[x] == "G", inw.neighbors(node_to_vaccinate)))
                k_info = len(inw.neighbors(node_to_vaccinate))
                if np.random.uniform() < zeta_info * (1 - n_G / k_info):
                        disease_states[node_to_vaccinate] = "V1"
                        susceptible.remove(node_to_vaccinate)
                        vaccinated_first.append(node_to_vaccinate)
                        N_vaccine_1 += 1


          # Case 7: Get vaccine 1 based on neighbourhood vaccination level
          # rate = zeta_epi * (1 - n_V1 / k_epi)
          # n_V1 is the vaccinated with vaccine 1
          # while k_epi is the total neighbor count on the epidemics layer
          elif event < p7:
            if len(susceptible) > 0:
                node_to_vaccinate = np.random.choice(susceptible)
                n_V1 = sum(1 for node in filter(lambda x: disease_states[x] == "V1", enw.neighbors(node_to_vaccinate)))
                k_epi = len(enw.neighbors(node_to_vaccinate))
                if np.random.uniform() < zeta_epi * (1 - n_V1 / k_epi):
                        disease_states[node_to_vaccinate] = "V1"
                        susceptible.remove(node_to_vaccinate)
                        vaccinated_first.append(node_to_vaccinate)
                        N_vaccine_1 += 1

      #print("N_infected", N_infected, "N_gossip", N_gossip)
      if N_infected == 0:
          corrected_frac = N_corrected / N
          vaccine_1_frac = N_vaccine_1 / N
          vaccine_2_frac = N_vaccine_2 /N
          recovered_frac = N_recovered / N
          rho_C.append(corrected_frac)
          rho_V1.append(vaccine_1_frac)
          rho_V2.append(vaccine_2_frac)
          rho_R.append(recovered_frac)
          #print("corrected_frac", corrected_frac, "recovered_frac", recovered_frac)

  if len(rho_C)>0 and len(rho_V1)>0 and len(rho_V2)>0and len(rho_R)>0:
      avg_rho_C = sum(rho_C) / len(rho_C)
      avg_rho_V1 = sum(rho_V1) / len(rho_V1)
      avg_rho_V2 = sum(rho_V2) / len(rho_V2)
      avg_rho_R = sum(rho_R) / len(rho_R)
  else:
      avg_rho_R = 0

  return avg_rho_C, avg_rho_V1, avg_rho_V2, avg_rho_R


In [21]:
n = 400  # Number of nodes

# Information Layer
gamma_i = 2.5
kmin_i = 3
ldeg_i = generate_degree_sequence(n, gamma_i, kmin_i)
print("Degree Sequence: ", ldeg_i)
inw = nx.configuration_model(ldeg_i)
ltre = assign_thresholds(inw, 0.05, 0.03)

# Epidemic Layer
gamma_e = 4.0
kmin_e = 3
ldeg_e = generate_degree_sequence(n, gamma_e, kmin_e)
print("Degree Sequence: ", ldeg_e)
enw = nx.configuration_model(ldeg_e)

Degree Sequence:  [20, 5, 3, 3, 3, 3, 6, 9, 5, 4, 3, 3, 3, 3, 4, 9, 3, 3, 3, 3, 11, 11, 4, 8, 4, 7, 4, 4, 4, 6, 4, 4, 4, 3, 3, 5, 4, 3, 5, 7, 4, 5, 3, 5, 3, 6, 4, 4, 9, 4, 4, 5, 3, 4, 8, 3, 5, 3, 3, 4, 5, 10, 5, 6, 3, 3, 4, 8, 3, 6, 3, 3, 5, 5, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 5, 6, 3, 5, 8, 3, 4, 4, 3, 3, 5, 4, 3, 4, 3, 3, 3, 3, 4, 7, 4, 4, 7, 7, 3, 3, 3, 3, 3, 3, 5, 3, 6, 6, 3, 3, 5, 6, 3, 6, 3, 3, 4, 3, 7, 3, 7, 5, 15, 4, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [10]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Set parameters
lam = 0.2
alp = 1
zeta_info = 0.1
zeta_epi = 0.1
beta_1 = 0.3
beta_2 = beta_1*(1-0.76)
beta_3 = beta_1*(1-0.94)
mu = 1
n_sample = 100

# Vary V1 percentage
V1_percentage_values = np.arange(0.1, 1.0, 0.1)

# Initialize the result array
results_rho_R = np.zeros(len(V1_percentage_values))

# Iterate over V1 percentage
for i in enumerate(V1_percentage_values):
        avg_rho_R = Vaccine_model(inw, ldeg_i, ltre, enw, ldeg_e, lam, alp, zeta_info, zeta_epi, beta_1, beta_2, beta_3, mu, n_sample)
        results_rho_R[i] = avg_rho_R

# Plot the results
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(V1_percentage_values, results_rho_R)
ax.set_xlabel('Fraction of Type 1 Vaccine Given', fontsize = 16)
ax.set_ylabel('Attack Rate', fontsize = 16)
#ax.set_yscale('log')  # Set y-axis to be logarithmic
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
ax.legend(fontsize = 16)
plt.show()
